In [2]:
import numpy as np
import pandas as pd
import tqdm

In [3]:
%%time
edges = pd.read_csv('./edges.csv')
vertices = pd.read_csv('./vertices.csv', index_col=0) 

C:\Users\lissrbay\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Wall time: 2.93 s


In [3]:
edges.head()

,id_1,id_2,value,n_transactions
0,878327,1133997,478035.238733,277.747437
1,707356,1341541,442189.669684,80.997950
2,169982,494074,353097.929209,287.789650
3,551010,979933,537749.674840,426.743337
4,76064,597023,418990.198382,287.789650


In [4]:
vertices.head()

,main_okved,region_code,company_type
id,,,
1,46.75,77,Limited
2,41.20,78,Limited
3,25.11,50,Limited
4,45.31,89,Limited
5,56.10,50,Limited


###  Немного информации о датасете

In [5]:
vertices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1534749 entries, 1 to 1534749
Data columns (total 3 columns):
main_okved      1534749 non-null float64
region_code     1534749 non-null int64
company_type    1534749 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 46.8+ MB


In [6]:
print("Число различных кодов деятельностей:", vertices['main_okved'].nunique())

Число различных кодов деятельностей: 1073


In [7]:
print("Число различных кодов регионов:", vertices['region_code'].nunique())

Число различных кодов регионов: 89


In [8]:
print("Число различных видов компаний:", vertices['company_type'].nunique())

Число различных видов компаний: 6


In [9]:
edges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4685644 entries, 0 to 4685643
Data columns (total 4 columns):
id_1              int64
id_2              int64
value             float64
n_transactions    float64
dtypes: float64(2), int64(2)
memory usage: 143.0 MB


In [10]:
e = edges.groupby('id_1').size()
e = e.reset_index()
e = e.set_index('id_1')
e2 = edges.groupby('id_2').size()
e2 = e2.reset_index()
e2 = e2.set_index('id_2')
e = e.join(e2, lsuffix='_')
e = e.fillna(0)
e = e['0_'] + e['0']
print("В среднем на каждую вершину у нас {} ребер.".format(int(e.mean())))
print("А медиана {} ребер.".format(int(e.median())))

В среднем на каждую вершину у нас 8 ребер.
А медиана 2 ребер.


### Добавим информацию, которую можно получить из информации о ребрах.

Преобразуем ребра так, чтобы можно было привести значения к логарифмированным.

In [11]:
max_v = edges['value'].max()
min_v = -edges['value'].min()
edges['value'] = edges['value'].apply(lambda x: x+min_v + 1) # чтобы привести к диапазону (1, inf) 
edges = edges.replace(-np.inf, 0)
edges = edges.replace(np.inf, 0)
edges = edges.replace(np.nan, 0)

Для каждой вершины запишем сумму потока денег и транзакций через нее.

In [12]:
vertices = vertices.join(edges.groupby('id_1')['n_transactions'].sum())
vertices = vertices.join(edges.groupby('id_2')['n_transactions'].sum(), lsuffix='1', rsuffix='2')
vertices.n_transactions1 = vertices.n_transactions1.fillna(0)
vertices.n_transactions2 = vertices.n_transactions2.fillna(0)
vertices['n_transactions'] = vertices['n_transactions1'] + vertices['n_transactions2']
vertices = vertices.drop(['n_transactions1', 'n_transactions2'], axis=1)
vertices.head()

,main_okved,region_code,company_type,n_transactions
id,,,,
1,46.75,77,Limited,2.000000
2,41.20,78,Limited,187.749487
3,25.11,50,Limited,207.791701
4,45.31,89,Limited,604.608179
5,56.10,50,Limited,535.494874


In [13]:
vertices = vertices.join(edges.groupby('id_1')['value'].sum())
vertices = vertices.join(edges.groupby('id_2')['value'].sum(), lsuffix='1', rsuffix='2')
vertices.value1 = vertices.value1.fillna(0)
vertices.value2 = vertices.value2.fillna(0)
vertices['value'] = vertices['value1'] + vertices['value2']
vertices = vertices.drop(['value1', 'value2'], axis=1)
vertices.head()

,main_okved,region_code,company_type,n_transactions,value
id,,,,,
1,46.75,77,Limited,2.000000,1.086125e+06
2,41.20,78,Limited,187.749487,1.267629e+06
3,25.11,50,Limited,207.791701,6.480210e+05
4,45.31,89,Limited,604.608179,2.037809e+06
5,56.10,50,Limited,535.494874,2.255042e+06


Прологарифмируем полученные значения.

In [14]:
vertices.value = vertices.value.apply(np.log)
vertices.n_transactions = vertices.n_transactions.apply(np.log)
vertices = vertices.replace(-np.inf, 0)
vertices = vertices.replace(np.inf, 0)

Сменим тип переменных на строковый, для упрощения работы с ними.

In [15]:
vertices['main_okved'] = vertices['main_okved'].astype(str) 
vertices['company_type'] = vertices['company_type'].astype(str) 
vertices['region_code'] = vertices['region_code'].astype(str) 
vertices['main_okved2'] = vertices['main_okved'].apply(lambda x: x.split('.')[0])

In [16]:
# ручное кодирование переменных (нужно для эмбеддингов)
#for column in ['main_okved','region_code', 'company_type']:
#    for value in tqdm.tqdm(vertices[column].unique()):
#        vertices[f'{column}_{value}'] = vertices[column].apply(lambda x: int(x == value)).astype('int8')

In [17]:
vertices.head()

,main_okved,region_code,company_type,n_transactions,value,main_okved2
id,,,,,,
1,46.75,77,Limited,0.693147,13.898127,46
2,41.2,78,Limited,5.235109,14.052659,41
3,25.11,50,Limited,5.336536,13.381678,25
4,45.31,89,Limited,6.404581,14.527386,45
5,56.1,50,Limited,6.283191,14.628679,56


### Добавим внешнюю информацию

In [18]:
regions = pd.read_csv('./regions.csv', index_col=0)
regions['code'] = regions['code'].astype(str)
regions = regions.set_index('code')
regions.head()

,reg_name,population,fed_district
code,,,
77,Москва,12506468,Центральный
50,Московская область,7503385,Центральный
23,Краснодарский край,5603420,Южный
78,Санкт-Петербург,5351935,Северо-Западный
66,Свердловская область,4325256,Уральский


In [19]:
okved = pd.read_csv('./okved.csv', index_col=0)
okved['code'] = okved['code'].astype(str)
okved.head()

,code,okved,letter
0,1,"СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО...",А
1,2,"СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО...",А
2,3,"СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО...",А
3,5,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,B
4,6,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,B


Добавляем инофрмацию о кодах специализации и региона.

In [20]:
vertices = vertices.join(okved.set_index('code'), on='main_okved2')
vertices['region_code'] = vertices['region_code'].astype(str)
vertices = vertices.join(regions, on='region_code')
vertices = vertices.drop(['reg_name', 'okved'], axis=1)
vertices.head()

,main_okved,region_code,company_type,n_transactions,value,main_okved2,letter,population,fed_district
id,,,,,,,,,
1,46.75,77,Limited,0.693147,13.898127,46,G,12506468.0,Центральный
2,41.2,78,Limited,5.235109,14.052659,41,F,5351935.0,Северо-Западный
3,25.11,50,Limited,5.336536,13.381678,25,С,7503385.0,Центральный
4,45.31,89,Limited,6.404581,14.527386,45,G,538547.0,Уральский
5,56.1,50,Limited,6.283191,14.628679,56,I,7503385.0,Центральный


Заполним пропуски в данных, которые отсутствуют в базе средним значением, или же значением из интернета(если номер региона просто устарел).

In [21]:
vertices.loc[(vertices['region_code']==str(99)), 'population'] = vertices['population'].mean()
vertices.loc[(vertices['region_code']==str(99)), 'fed_district'] = 'Мир'
vertices.loc[(vertices['region_code']==str(81)), 'population'] = 2623122
vertices.loc[(vertices['region_code']==str(81)), 'fed_district'] = 'Приволжский'
vertices.loc[(vertices['region_code']==str(85)), 'population'] = 2404195
vertices.loc[(vertices['region_code']==str(85)), 'fed_district'] = 'Сибирский'
vertices.loc[(vertices['region_code']==str(0)), 'population'] = vertices['population'].mean()
vertices.loc[(vertices['region_code']==str(0)), 'fed_district'] = 'Беларусь'
vertices['fed_district'] = vertices['fed_district'].astype(str)
vertices['letter'] = vertices['letter'].astype(str)
vertices['population'] = vertices['population'].apply(np.log)

Уберем значение кода региона, так как оно дублируется значением численности населения и при этом не определяет порядок т.е. заведомо хуже.

In [22]:
vertices = vertices.drop(['region_code'], axis=1)

А вот следующий шаг был ошибкой :с

Убираем вершины, в которые нет ребер в графе.

In [23]:
ex_vertices = list(set(edges['id_1'].values).union(edges['id_2'].values))
vertices = vertices.loc[ex_vertices]

In [24]:
vertices.head()

,main_okved,company_type,n_transactions,value,main_okved2,letter,population,fed_district
id,,,,,,,,
1,46.75,Limited,0.693147,13.898127,46,G,16.341757,Центральный
2,41.2,Limited,5.235109,14.052659,41,F,15.492969,Северо-Западный
3,25.11,Limited,5.336536,13.381678,25,С,15.830865,Центральный
4,45.31,Limited,6.404581,14.527386,45,G,13.196630,Уральский
5,56.1,Limited,6.283191,14.628679,56,I,15.830865,Центральный
